In [1]:
! pip install -U accelerate
! pip install -U transformers
! pip install evaluate
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 39.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.2 MB/s eta 0:00:00


# Ch6.2-Training tokenizer from an old one

If we want to make model work on a new langauge which has English character but corpus is very different to what it was trained on e.g.

We want to train model on specialized English language (Python code)

In [2]:
from datasets import load_dataset

# This can take a few minutes to load, so grab a coffee or tea while you wait!
raw_datasets = load_dataset("code_search_net", "python")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [3]:
raw_datasets["train"]


Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

The text that we want to train on is in `whole_func_string`

In [9]:
def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]

training_corpus = get_training_corpus()

Even though we are going to train a new tokenizer, it’s a good idea to do this to avoid starting entirely from scratch. This way, we won’t have to specify anything about the tokenization algorithm or the special tokens we want to use; our new tokenizer will be exactly the same as GPT-2, and the only thing that will change is the vocabulary, which will be determined by the training on our corpus.

In [5]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
' '.join(tokens) # old tokenizer is not good at tokenizing Python code

'def Ġadd _ n umbers ( a , Ġb ): Ċ Ġ Ġ Ġ Ġ""" Add Ġthe Ġtwo Ġnumbers Ġ` a ` Ġand Ġ` b ` ." "" Ċ Ġ Ġ Ġ Ġreturn Ġa Ġ+ Ġb'

In [11]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [12]:
tokens = tokenizer.tokenize(example)
' '.join(tokens) # new tokenizer is better

'def Ġadd _ numbers ( a , Ġb ): ĊĠĠĠ Ġ""" Add Ġthe Ġtwo Ġnumbers Ġ` a ` Ġand Ġ` b `.""" ĊĠĠĠ Ġreturn Ġa Ġ+ Ġb'

In [13]:
print(len(tokens))
print(len(old_tokenizer.tokenize(example))) # Tokenization by new is more compact.
# It has understood that  ĊĠĠĠ = indentation, Ġ""" = tripple quotes for docstring
# and groups them together instead of tokenizing separately

27
36


In [14]:
# Saving the tokenizer
tokenizer.save_pretrained("code-search-net-tokenizer")

# We can also push to hub so that it is saved and easy to use

('code-search-net-tokenizer/tokenizer_config.json',
 'code-search-net-tokenizer/special_tokens_map.json',
 'code-search-net-tokenizer/vocab.json',
 'code-search-net-tokenizer/merges.txt',
 'code-search-net-tokenizer/added_tokens.json',
 'code-search-net-tokenizer/tokenizer.json')